In [1]:
import requests
from bs4 import BeautifulSoup
import duckdb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import regex as re
import plotly.express as px

### Dictionary

This dictionary of states to acronyms and vise versa is useful as one of our datasets lists states with their 2 letter acronym and our graph makes use of that acronym. 

In [2]:
code = {'Alabama': 'AL',
        'Alaska': 'AK',
        'Arizona': 'AZ',
        'Arkansas': 'AR',
        'California': 'CA',
        'Colorado': 'CO',
        'Connecticut': 'CT',
        'Delaware': 'DE',
        'District of Columbia': 'DC',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Hawaii': 'HI',
        'Idaho': 'ID',
        'Illinois': 'IL',
        'Indiana': 'IN',
        'Iowa': 'IA',
        'Kansas': 'KS',
        'Kentucky': 'KY',
        'Louisiana': 'LA',
        'Maine': 'ME',
        'Maryland': 'MD',
        'Massachusetts': 'MA',
        'Michigan': 'MI',
        'Minnesota': 'MN',
        'Mississippi': 'MS',
        'Missouri': 'MO',
        'Montana': 'MT',
        'Nebraska': 'NE',
        'Nevada': 'NV',
        'New Hampshire': 'NH',
        'New Jersey': 'NJ',
        'New Mexico': 'NM',
        'New York': 'NY',
        'North Carolina': 'NC',
        'North Dakota': 'ND',
        'Ohio': 'OH',
        'Oklahoma': 'OK',
        'Oregon': 'OR',
        'Pennsylvania': 'PA',
        'Rhode Island': 'RI',
        'South Carolina': 'SC',
        'South Dakota': 'SD',
        'Tennessee': 'TN',
        'Texas': 'TX',
        'Utah': 'UT',
        'Vermont': 'VT',
        'Virginia': 'VA',
        'Washington': 'WA',
        'West Virginia': 'WV',
        'Wisconsin': 'WI',
        'Wyoming': 'WY'}
reversed_code = {}
for state_name, abbreviation in code.items():
    reversed_code[abbreviation] = state_name

## State Gun Ownership Data

To gather this data we were able to scrape it from the website. Below is a simple scraping method we used to get this data into a dataframe form. 

In [3]:
url = "https://wisevoter.com/state-rankings/gun-ownership-by-state/#alabama"

response = requests.get(url)

lst = []

soup = BeautifulSoup(response.text, "html.parser")

table = soup.find("table")

for row in table.find_all("tr"):
    for cell in row.find_all("td"):
        lst.append(cell.get_text(strip=True))


In [4]:
data = np.array(lst).reshape(-1,6)
stategundf = pd.DataFrame(data, columns=["Order", "State", "gun_ownership_rate",\
                                         "Registered Gun Rate", "Gun Death Rate","Party"])

stategundf.head()

,Order,State,gun_ownership_rate,Registered Gun Rate,Gun Death Rate,Party
0,1,Montana,65.7%,33.2 per 1k,20.9 per 100k,Republican
1,2,Wyoming,60.7%,245.8 per 1k,25.9 per 100k,Republican
2,3,West Virginia,60.6%,92.8 per 1k,18.1 per 100k,Republican
3,4,Idaho,57.8%,40.2 per 1k,17.6 per 100k,Republican
4,5,Alaska,57.2%,38.5 per 1k,23.5 per 100k,Republican


To clean this dataframe, we removed all of the % signs and turned the percentages into decimals. The dataset also labeled numbers as “per 1000” and “per 100k” so we got stripped those and turned the numbers into floats. We then divided by what the number was so we had the same rate as before. We also mapped Republican to 1 and Democrat to 0 so we could run regression models on them in the future. 


In [5]:
#Turning strings of percentages of Gun ownerships rates to floats
stategundf["gun_ownership_rate"] = \
stategundf["gun_ownership_rate"].str.rstrip("%").astype(float)/100
# The current format of the registered gun rate is
#X per 1k and is a string we are changing it to a float
stategundf["Registered Gun Rate"] = \
stategundf["Registered Gun Rate"].str.rstrip(" per 1k").astype(float)/1000
# The current format of the Gun death gun rate is 
#X per 100k and is a string we are changing it to a float
stategundf["Gun Death Rate"] = \
stategundf["Gun Death Rate"].str.rstrip(" per 100k").astype(float)/100000
# Turning Republican and Democrat Values into a binary 1 or 0
stategundf['Party_Binary'] = stategundf['Party'].map({'Republican':1, 'Democrat':0})
#drop order
stategundf =stategundf.drop(["Order"], axis =1)

# show df
stategundf.head()

,State,gun_ownership_rate,Registered Gun Rate,Gun Death Rate,Party,Party_Binary
0,Montana,0.657,0.0332,0.000209,Republican,1
1,Wyoming,0.607,0.2458,0.000259,Republican,1
2,West Virginia,0.606,0.0928,0.000180,Republican,1
3,Idaho,0.578,0.0402,0.000176,Republican,1
4,Alaska,0.572,0.0385,0.000235,Republican,1


## Congressional District

In order to get the neccesary information for the congressional districts we also had to scrape the data. This website only had text so turning the website into a json file and extracting the text works. 

For the congressional data frame, the district names were in a format that made it impossible to merge. All Districts were written as “Congressional District 1 (116th Congress), Alabama” which was not consistent with the other data frames. To fix this we used regex to select the first digit up until the white space and then the last word. We split those two into different columns. We also dropped 2 columns that served no purpose. 

In [6]:
url = \
"https://api.census.gov/data/2019/acs/acs1?get=NAME,B01001_001E,B19013_001E&for=congressional%20district:*"
response = requests.get(url)
data = response.json()
data.pop(0)
headers = ["district","population","median_income","drop1", "drop2"]
congressional = pd.DataFrame(data, columns = headers)
congressional = congressional.drop(columns = ["drop1","drop2"])
congressional.head()
def check_for_string(substring, string):
    match = re.search(substring, string)
    if match:
        return (match.group())
congressional["congressional_district"] = \
congressional["district"].apply(lambda x: check_for_string("\d+", x))
congressional["congressional_district"] = \
congressional["congressional_district"].astype(float)
congressional["state"] = \
congressional["district"].apply(lambda x: check_for_string(r",\s(\w+\s?\w+)$", x))
congressional["state"] = congressional["state"].str[2:]
congressional = congressional.drop(columns = "district")
congressional.loc[congressional["congressional_district"] == 116.0, "congressional_district"] = 1
congressional["median_income"] = congressional["median_income"].astype(float)/1000

## Total Gun Deaths

The total gun deaths files was a csv that was downloaded from kaggle. This made the proccess of loading it into this notebook to be easy.

Since we downloaded his dataframe from Kaggle it did not require much cleaning. However to make it more manageable we renamed columns, dropped unnecessary columns, and changed the congressional_district to a float instead of a string.


In [7]:
# Loading in the dataframe
total_gun_violence = pd.read_csv("gun-violence-data.csv")

#Renaming Columns
total_gun_violence = \
total_gun_violence.rename(columns={'n_injured': 'victims_injured',\
                    'n_killed': 'victims_killed', 'date': 'incident_date'})
#dropping unused columns to make the dataset more manageable
total_gun_violence = total_gun_violence.drop(columns = [
    "incident_id", "city_or_county", "address",
    "incident_url", "source_url", "incident_url_fields_missing",
    "participant_age", "participant_age_group", "participant_gender",
    "participant_name", "participant_status",
    "participant_type", "sources", "state_house_district", "state_senate_district", "gun_type",
    "incident_characteristics", "location_description", "gun_stolen", "longitude", "latitude",
    "notes", "participant_relationship", "n_guns_involved"
])
total_gun_violence["congressional_district"] = \
total_gun_violence["congressional_district"].astype(float)
total_gun_violence["state"] = total_gun_violence["state"].astype(str)

total_gun_violence.head()



,incident_date,state,victims_killed,victims_injured,congressional_district
0,2013-01-01,Pennsylvania,0,4,14.0
1,2013-01-01,California,1,3,43.0
2,2013-01-01,Ohio,1,3,9.0
3,2013-01-05,Colorado,4,0,6.0
4,2013-01-07,North Carolina,2,2,6.0


## Population Density

The population Density file was downloaded as a CSV and loaded into python. In order to clean the  dataframe we first split up the two-letter state code and the district number. We then used a dictionary to match each code to its state and made a function to turn each district number which was a string into a float.


In [8]:
specific = pd.read_csv("2010_districts.csv")

def split(statedistrict):
    parts = [statedistrict[0:2],statedistrict[3:]]
    state = parts[0]
    district = parts[1]

    if district == "AL":
        district = 1.0
    else:
        district= float(district)
    
    return state, district

specific['state'], specific['district'] = zip(*specific['CD'].apply(split))

specific["state"] = \
specific["state"].map(reversed_code)
specific["percent_r"] = specific["mccain08"]/ (specific["obama08"] + specific["mccain08"] )

specific = \
specific[["state", "district", "Very low density","Low density" \
          ,"Medium density" ,"High density", "percent_r"]]
specific = specific.rename(columns={ "Very low density": "Very_low_density", "Low density": "Low_density",
    "Medium density": "Medium_density", "High density": "High_density", })
specific.head()

,state,district,Very_low_density,Low_density,Medium_density,High_density,percent_r
0,Alaska,1.0,0.502945,0.196026,0.202034,0.098995,0.608247
1,Alabama,1.0,0.402159,0.389192,0.196183,0.012465,0.610000
2,Alabama,2.0,0.552118,0.281047,0.161855,0.004980,0.636364
3,Alabama,3.0,0.495272,0.404430,0.094354,0.005944,0.565657
4,Alabama,4.0,0.754078,0.236419,0.009504,0.000000,0.767677


## Merging Data

In order to create a useful data set we had to  group by state and district and then merge on those two columns. The below code merges, total_gun_violence, party_district,congressional and population density. We then dropped columns that were either repeating or not needed in our regression models. We then added columns for killed per capita, injured per capita, and total victims per capita as those were our primary metrics and will serve as our y values in regression models. Together this dataframe is grouped by congressional district and has all of the needed variables for our model.

In [9]:
merged_data = total_gun_violence.merge(congressional, 
                                       left_on=['state', 'congressional_district'], 
                                       right_on=['state', 'congressional_district'], 
                                       how='inner')
merged_data["incident_date"] = pd.to_datetime(merged_data["incident_date"])





merged_data = \
merged_data.merge(stategundf, left_on = "state", right_on = "State", how = "right")

merged_data =\
merged_data.drop(columns = ["Gun Death Rate", "Registered Gun Rate", "State"])


merged_data = merged_data.merge(specific, 
                                       left_on=['state', 'congressional_district'], 
                                       right_on=['state', 'district'], 
                                       how='inner')
nan_rows = merged_data[merged_data.isna().any(axis=1)]
print(nan_rows.shape)

merged_data["population"] = merged_data["population"].astype(float)
merged_data["victims_injured"] =merged_data["victims_injured"].astype(float)
merged_data["victims_killed"] = merged_data["victims_killed"].astype(float)
merged_data["district_binary_party"] = np.where(merged_data["percent_r"] > 0.5, 1, 0)

sortedbefore = \
merged_data[(merged_data["incident_date"].dt.year <= 2016) & (merged_data["incident_date"].dt.year > 2013)]
sortedafter = merged_data[merged_data["incident_date"].dt.year > 2016]

merged_data_combined = duckdb.sql("""
SELECT 
    state, 
    congressional_district, 
    population, 
    gun_ownership_rate, 
    median_income, 
    SUM(victims_killed) AS total_killed, 
    Very_low_density, 
    Low_density, 
    Medium_density, 
    High_density,  
    percent_r, 
    district_binary_party, 
    SUM(victims_injured) AS total_injured 
FROM 
    merged_data 
GROUP BY 
    state, 
    congressional_district, 
    Very_low_density, 
    Low_density,
    Medium_density, 
    High_density, 
    percent_r,
    median_income, 
    population,
    district_binary_party, 
    gun_ownership_rate
""").df()

merged_data_train = duckdb.sql("""
SELECT 
    state, 
    congressional_district, 
    population, 
    gun_ownership_rate,
    median_income,
    district_binary_party, 
    SUM(victims_killed) AS total_killed, 
    Very_low_density, 
    Low_density,
    Medium_density, 
    High_density, 
    percent_r,
    SUM(victims_injured) AS total_injured 
FROM 
    sortedbefore
GROUP BY 
    state, 
    congressional_district, 
    district_binary_party, 
    Very_low_density, 
    Low_density,
    Medium_density, 
    High_density, 
    percent_r,
    median_income, 
    population, 
    gun_ownership_rate
""").df()


merged_data_test = duckdb.sql("""
SELECT 
    state, 
    congressional_district, 
    population, 
    gun_ownership_rate,
    district_binary_party, 
    median_income, 
    SUM(victims_killed) AS total_killed, 
    Very_low_density, 
    Low_density,
    Medium_density, 
    High_density, 
    percent_r,
    SUM(victims_injured) AS total_injured 
FROM 
    sortedafter
GROUP BY 
    state, 
    congressional_district, 
    district_binary_party, 
    Very_low_density, 
    Low_density,
    Medium_density, 
    High_density, 
    percent_r,
    median_income, 
    population, 
    gun_ownership_rate
""").df()

def add_columns(df):
    df["killed_per_capita"] = df["total_killed"] / df["population"]
    df["injured_per_capita"] = df["total_injured"] / df["population"]
    df["total_victims"] = df["total_killed"] + df["total_injured"]
    df["total_victims_per_capita"] = df["killed_per_capita"] + df["injured_per_capita"]
add_columns(merged_data_train)
add_columns(merged_data_test)
add_columns(merged_data_combined)



(0, 16)


In [10]:
merged_data['incident_date'] = pd.to_datetime(merged_data['incident_date'])

merged_data['month'] = merged_data['incident_date'].dt.month
merged_data['year'] = merged_data['incident_date'].dt.year


merged_data['total_victims'] = (merged_data['victims_killed'] + merged_data['victims_injured'])
merged_data['total_victims_population'] = (merged_data['total_victims'])/merged_data['population']
merged_data['code'] = merged_data['state'].map(code)

result_df = merged_data.groupby(['code', 'year', 'month','state'])['total_victims_population'].sum().reset_index()

result_df = result_df.sort_values(by=['year', 'month'])


In [11]:



state_data_train= duckdb.sql("""SELECT 
    state, 
    SUM(victims_injured) AS total_injured,
    SUM(victims_killed) AS total_killed,
    SUM(population) AS population, 
    gun_ownership_rate, Party,
    Party_Binary
    FROM sortedbefore 
    GROUP BY state,
    Party, 
    gun_ownership_rate, 
    Party_Binary""").df()
add_columns(state_data_train)

state_data_test = duckdb.sql("""SELECT 
    state,
    SUM(victims_injured) AS total_injured, 
    SUM(victims_killed) AS total_killed, 
    SUM(population) AS population, 
    gun_ownership_rate, 
    Party, 
    Party_Binary 
    FROM sortedafter 
    GROUP BY 
    state,
    Party, 
    gun_ownership_rate, 
    Party_Binary""").df()

add_columns(state_data_test)



state_data =duckdb.sql("""SELECT 
    state,
    SUM(victims_injured) AS total_injured, 
    SUM(victims_killed) AS total_killed, 
    SUM(population) AS population, 
    gun_ownership_rate, 
    Party, 
    Party_Binary 
    FROM merged_data 
    GROUP BY 
    state,
    Party, 
    gun_ownership_rate, 
    Party_Binary""").df()

add_columns(state_data)


In [12]:
merged_data_train.to_csv("merged_train.csv")
merged_data_test.to_csv("merged_test.csv")
state_data_train.to_csv("state_data_train.csv")
state_data_test.to_csv("state_data_test.csv")
merged_data_combined.to_csv("merged_data_combined.csv")
state_data.to_csv("state_data.csv")
result_df.to_csv("graph_data.csv")